In [1]:
from src.models.loader import DataModule
from src.models.heterolgn import HeteroLGN

import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.utils import to_torch_coo_tensor
from torchmetrics import RetrievalPrecision, RetrievalRecall

In [2]:
dm = DataModule(batch_size=128)
x = next(iter(dm.train_dataloader()))
x

HeteroData(
  recipe={
    x=[1886, 9],
    id=[1886],
    dst_pos_index=[128],
    dst_neg_index=[128]
  },
  user={
    x=[1300, 0],
    id=[1300],
    src_index=[128]
  },
  (user, reviews, recipe)={
    edge_index=[2, 1657],
    edge_attr=[1657],
    edge_label=[1811790],
    edge_label_index=[2, 1811790],
    input_id=[128]
  },
  (recipe, rev_reviews, user)={
    edge_index=[2, 1874],
    edge_attr=[1874]
  }
)

In [3]:
model = HeteroLGN(num_users=dm.num_users, num_recipes=dm.num_recipes)
model

HeteroLGN(
  (usr_embedding): Embedding(226570, 64)
  (rec_embedding): Embedding(231637, 64)
  (conv): HeteroLGConv()
  (recall): RetrievalRecall()
  (precision): RetrievalPrecision()
)

In [4]:
usr_out, rec_out = model.forward(x.id_dict, x.edge_index_dict)

In [5]:
import torch.nn.functional as F

In [6]:
scores = usr_out @ rec_out.T
indicies = torch.arange(scores.size(0))[:, None].expand_as(scores)
target = to_torch_coo_tensor(
    x["user", "reviews", "recipe"].edge_index,
    size=(usr_out.size(0), rec_out.size(0))
    # Current limitations with metrics require dense version :(
).to_dense()
    

In [15]:
target.sum(axis=0).mean()

tensor(0.8786)

In [30]:
from torchmetrics import RetrievalPrecision
p2 = RetrievalPrecision(k=10)
p2(scores[1], target[1], indexes=indicies[1])

tensor(0.)

In [14]:
usr_out.shape, rec_out.shape

(torch.Size([1292, 64]), torch.Size([1841, 64]))

In [10]:
scores.shape, target.shape, indicies.shape

(torch.Size([1292, 1841]), torch.Size([1292, 1292]), torch.Size([1292, 1841]))

In [31]:
target[1].nonzero()

tensor([[ 86],
        [152],
        [260],
        [262],
        [263],
        [264]])

In [32]:
scores[1].topk(10).indices

tensor([  2,  22, 615,  47, 130,   0, 156, 193,  26, 245])